<p align="center">
    <a href="https://predict-idlab.github.io/landmarker">
        <img alt="landmarker" src="https://raw.githubusercontent.com/predict-idlab/landmarker/main/docs/_static/images/logo.svg" width="66%">
    </a>
</p>

# Demonstration of Pipeline for Training/Evaluation Landmark Detection (ISBI 2015 Cephalometric X-ray Dataset)

In this tutorial, we will demonstrate the use of the Pipline Module in `landmarker` library, which
allows for training and evaluating landmark detection algorithms in a few lines of code. We will use 
the ISBI 2015 Cephalometric X-ray dataset to train and evaluate the SpatialConfiguration model.
The ISBI 2015 Cephalometric X-ray dataset is a dataset of 2D cephalometric X-rays. The dataset 
contains 400 images, each with 19 landmarks annotated. The dataset is split into a training set of 
150 images and two test sets of 150 and 100 images respectively.

<a target="_blank" href="https://colab.research.google.com/github/predict-idlab/landmarker/examples/pipeline_scn_isbi2015.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# import everyting from data module which is located ../src/data.py
import sys
sys.path.append('../src')

In [2]:
# !python -c "import landmarker" || pip install landmarker

## Downloading the dataset
The `landmarker` library provides a utility function to download the ISBI 2015 Cephalometric X-ray
dataset. The dataset will be downloaded to the `data` folder in the current working directory.

In [3]:
from landmarker.datasets.cepha400 import get_cepha_dataset
path_dir = "data/"
(image_paths_train, image_paths_test1, image_paths_test2,
 landmarks_train, landmarks_test1, landmarks_test2,
 pixel_spacings_train, pixel_spacings_test1, pixel_spacings_test2,
 ) = get_cepha_dataset(path_dir=path_dir, junior=False, cv=False)

## Pipelines in `landmarker`
The `landmarker` library provides a `Pipeline` class that allows for training and evaluating landmark
detection algorithms in a few lines of code. The `Pipeline` class is a wrapper around different
components of the library. There are five main types of pipelines, which are distinguished by our
own categorizion of landmark detection algorithms. See the figure below for categorization of
landmark detection algorithms.

<p align="center">
    <img alt="landmarker" src="../docs/_static/images/categorization_detection_approaches.svg" width="50%">
</p>

We can divide landmark detection algorithms into two main categories: *Heatmap-based* and 
*Coordinate-based*. Heatmap-based algorithms predict a heatmap for each landmark, this heatmap is
than decoded to obtain the landmark location. Coordinate-based algorithms directly predict the
landmark location. Heatmap-based algorithms can be further divided into *Direct* and *Indirect*
approaches. In direct heatmap-based approaches, a heatmap is generated for each landmark following 
a probability distrubion (e.g., the bivariate gaussian distribution), this heatmap is than used as 
target for the network. In indirect heatmap-based approaches, the model predicts an intermediate
heatmap, which is than decoded (by a differentiable function) to obtain the landmarks. The Direct 
Heatmap-based algorithms can be further divided into *Static* and *Adaptive* approaches. In static
heatmap-based approaches, the heatmap is generated using a fixed probability distribution, i.e.,
the parameters of the probability distribution are constant the entire learning procdeure. In 
adaptive heatmap-based approaches, the parameters of the probability distribution are learned or
are adapted by a scheduler algorithm. The Indirect Heatmap-based algorithms can be further divided
into Uncertainty aware and Uncertainty unaware approaches. In Uncertainty aware approaches, the
algorithm decodes from the heamtmap the uncertainty besides the landmark location. In Uncertainty
unaware approaches, the algorithm only decodes the landmark location from the heatmap.

The `landmarker` library provides a `Pipeline` class for each of the five categories of landmark
detection algorithms. Each `Pipeline` class has a `fit`, `predict`, and `evaluate` method. 
The `fit` method is used to train the model, the `predict` method is used to predict landmarks on
new images, and the `evaluate` method is used to evaluate the model on a test set and give additional
information about the model performance.

The five `Pipeline` classes are:
- `StaticHeatmapRegressionPipeline`: Static Heatmap-based algorithms
- `AdaptiveHeatmapRegressionPipeline`: Adaptive Heatmap-based algorithms
- `IndirectUncertaintyAwareHeatmapRegressionPipeline`: Uncertainty aware Indirect Heatmap-based algorithms
- `IndirectUncertaintyUnawareHeatmapRegressionPipeline`: Uncertainty unaware Indirect Heatmap-based algorithms
- `CoordinateRegressionPipeline`: Coordinate-based algorithms

In [4]:
import torch
from landmarker.heatmap.decoder import coord_argmax
from monai.transforms import (Compose, RandAffined, RandGaussianNoised, RandStdShiftIntensityd,
                              RandScaleIntensityd, RandAdjustContrastd, RandHistogramShiftd)
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 1000
lr = 0.0001
batch_size = 4
cache_ds = True
dim_img = (512, 512)

## Static Heatmap Regression

In [5]:
from landmarker.pipeline import StaticHeatmapRegressionPipeline

train_config = {
    "criterion": "cross_entropy",
    "optimizer": "adam",
    "lr": lr,
    "batch_size": batch_size,
    "epochs": epochs,
    "lr_scheduler": True,
}


staticHeatmapRegressionPipeline = StaticHeatmapRegressionPipeline(
    sigmas=3.0, model="ProbSpatialConfigurationNet", in_channels_img=1, nb_landmarks=19, train_config=train_config,
    heatmap_decoder=coord_argmax, verbose=True, device=device)

fn_keys = ('image', 'seg')
spatial_transformd = [RandAffined(fn_keys, prob=1,
                        rotate_range=(-np.pi/12, np.pi/12),
                        translate_range=(-10, 10),
                        scale_range=(-0.1, 0.1),
                        shear_range=(-0.1, 0.1)
                        )]

composed_transformd = Compose(spatial_transformd +
                            [RandGaussianNoised(('image', ), prob=0.1, mean=0, std=0.005),  # Add gaussian noise
                        RandStdShiftIntensityd(('image', ), prob=0.1, factors=0.1),  # Add random intensity shift
                        RandScaleIntensityd(('image', ), factors=0.25, prob=0.1),  # Add random intensity scaling
                        RandAdjustContrastd(('image', ), prob=0.1, gamma=(0.5,2)),  # Randomly adjust contrast
                        RandHistogramShiftd(('image', ), prob=0.1),  # Randomly shift histogram
                        ])

staticHeatmapRegressionPipeline.fit(image_paths_train, landmarks_train,
                                    transform=composed_transformd, cache_ds=cache_ds)

Reading, normalizing and resizing 150 images and landmarks...


  0%|          | 0/150 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 150/150 [00:03<00:00, 44.88it/s]


Creating heatmaps for 150 images... (batch size: 32)


100%|██████████| 8/8 [00:00<00:00, 14.28it/s]


Epoch 1/1000 - Train loss: 0.48456608 - Val loss: 0.07820681


...

Epoch 119/1000 - Train loss: 0.00886878 - Val loss: 0.00983945
INFO: Early stopping counter                     18 of 20                         [Best Val Loss: 0.00978774,                             Val Loss: 0.00983945]


100%|██████████| 8/8 [00:00<00:00, 15.05it/s]


Epoch 120/1000 - Train loss: 0.00896552 - Val loss: 0.00984456
INFO: Early stopping counter                     19 of 20                         [Best Val Loss: 0.00978774,                             Val Loss: 0.00984456]


100%|██████████| 8/8 [00:00<00:00, 13.23it/s]


Epoch 121/1000 - Train loss: 0.00898887 - Val loss: 0.00984438
INFO: Early stopping counter                     20 of 20                         [Best Val Loss: 0.00978774,                             Val Loss: 0.00984438]
INFO: Early stopping DONE!
INFO: Best Val Loss: 0.00978774
Loading best model...


In [6]:
results_static = staticHeatmapRegressionPipeline.evaluate(image_paths_test1, landmarks_test1, pixel_spacings_test1)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 150/150 [00:02<00:00, 58.71it/s]

Detection report:
1# Point-to-point error (PE) statistics:
Class               Mean      Median    Std       Min       Max       
----------------------------------------------------------------------
Landmark 0          1.15      1.02      1.00      0.02      9.68      
Landmark 1          1.98      1.28      1.94      0.13      10.14     
Landmark 2          2.09      1.75      1.46      0.33      8.32      
Landmark 3          2.88      1.76      2.53      0.13      11.37     
Landmark 4          2.52      2.07      1.75      0.20      10.08     
Landmark 5          1.73      1.35      1.30      0.10      6.46      
Landmark 6          1.28      1.12      0.83      0.06      4.11      
Landmark 7          1.12      0.88      0.90      0.06      6.62      
Landmark 8          1.07      0.87      0.84      0.05      8.02      
Landmark 9          3.88      3.78      2.22      0.29      10.38     
Landmark 10         1.31      1.11      0.97      0.03      6.26      
Landmark 11       

## Adaptive Heatmap Regression

In [7]:
from landmarker.pipeline import AdaptiveHeatmapRegressionPipeline

train_config = {
    "criterion": "l2",
    "optimizer": "sgd",
    "lr": 1e-6,
    "batch_size": 1,
    "epochs": epochs,
    "lr_scheduler": True,
}


fn_keys = ('image', )
spatial_transformd = [RandAffined(fn_keys, prob=1,
                        rotate_range=(-np.pi/12, np.pi/12),
                        translate_range=(-10, 10),
                        scale_range=(-0.1, 0.1),
                        shear_range=(-0.1, 0.1)
                        )]

composed_transformd = Compose(spatial_transformd +
                            [RandGaussianNoised(('image', ), prob=0.1, mean=0, std=0.005),  # Add gaussian noise
                        RandStdShiftIntensityd(('image', ), prob=0.1, factors=0.1),  # Add random intensity shift
                        RandScaleIntensityd(('image', ), factors=0.25, prob=0.1),  # Add random intensity scaling
                        RandAdjustContrastd(('image', ), prob=0.1, gamma=(0.5,2)),  # Randomly adjust contrast
                        RandHistogramShiftd(('image', ), prob=0.1),  # Randomly shift histogram
                        ])

adaptiveHeatmapRegressionPipeline = AdaptiveHeatmapRegressionPipeline(
    sigmas=3.0, alpha=5.0, model="OriginalSpatialConfigurationNet", in_channels_img=1, nb_landmarks=19,
    train_config=train_config, heatmap_decoder=coord_argmax, verbose=True, device=device, dim_img=dim_img,
    gamma=100
)
adaptiveHeatmapRegressionPipeline.fit(image_paths_train, landmarks_train,
                                        transform=composed_transformd, cache_ds=cache_ds)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 30/30 [00:01<00:00, 25.07it/s]


Epoch 1/1000 - Train loss: 2520.17437948 - Val loss: 2497.31059570


100%|██████████| 30/30 [00:01<00:00, 22.60it/s]


Epoch 2/1000 - Train loss: 2471.79089559 - Val loss: 2446.74260254
INFO: Val_Loss improved (2497.31059570 --> 2446.74260254). Saving model ...


...

Epoch 130/1000 - Train loss: 1212.96040141 - Val loss: 1444.55605876
INFO: Early stopping counter                     18 of 20                         [Best Val Loss: 1435.54129232,                             Val Loss: 1444.55605876]


100%|██████████| 30/30 [00:01<00:00, 24.78it/s]


Epoch 131/1000 - Train loss: 1225.11510468 - Val loss: 1443.43215332
INFO: Early stopping counter                     19 of 20                         [Best Val Loss: 1435.54129232,                             Val Loss: 1443.43215332]


100%|██████████| 30/30 [00:01<00:00, 21.85it/s]


Epoch 132/1000 - Train loss: 1215.74841410 - Val loss: 1445.45094808
INFO: Early stopping counter                     20 of 20                         [Best Val Loss: 1435.54129232,                             Val Loss: 1445.45094808]
INFO: Early stopping DONE!
INFO: Best Val Loss: 1435.54129232
Loading best model...


In [8]:
results_adaptive = adaptiveHeatmapRegressionPipeline.evaluate(image_paths_test1, landmarks_test1, pixel_spacings_test1)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 150/150 [00:04<00:00, 34.66it/s]


Detection report:
1# Point-to-point error (PE) statistics:
Class               Mean      Median    Std       Min       Max       
----------------------------------------------------------------------
Landmark 0          0.72      0.54      1.23      0.03      11.49     
Landmark 1          1.71      0.93      1.97      0.03      11.26     
Landmark 2          1.94      1.50      1.53      0.07      9.10      
Landmark 3          2.98      1.87      2.56      0.05      10.33     
Landmark 4          2.24      1.82      1.51      0.14      7.10      
Landmark 5          1.70      1.23      1.49      0.03      7.45      
Landmark 6          1.06      0.82      0.84      0.08      4.99      
Landmark 7          1.04      0.78      1.04      0.06      8.76      
Landmark 8          0.96      0.83      0.71      0.08      4.37      
Landmark 9          3.64      3.34      2.14      0.06      9.09      
Landmark 10         0.89      0.66      0.97      0.03      8.65      
Landmark 11       

## Indirect Uncertainty Aware Heatmap Regression

In [9]:
from landmarker.pipeline import IndirectUncertaintyAwareHeatmapRegressionPipeline

train_config = {
    "criterion": "multivariate_gaussian_nllloss",
    "optimizer": "adam",
    "lr": lr,
    "batch_size": batch_size,
    "epochs": epochs,
    "lr_scheduler": True,
}

indirectUncertaintyAwareHeatmapRegressionPipeline = IndirectUncertaintyAwareHeatmapRegressionPipeline(
    differentiable_decoder="softargmax", model="CoordConvUNet", in_channels_img=1, nb_landmarks=19,
    train_config=train_config, heatmap_decoder=coord_argmax, verbose=True, device=device)
indirectUncertaintyAwareHeatmapRegressionPipeline.fit(image_paths_train, landmarks_train,
                                        transform=composed_transformd, cache_ds=cache_ds)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 8/8 [00:00<00:00,  9.28it/s]


Epoch 1/1000 - Train loss: 8.85510486 - Val loss: 7.59643334


100%|██████████| 8/8 [00:00<00:00,  9.53it/s]


Epoch 2/1000 - Train loss: 6.91894143 - Val loss: 6.12938422
INFO: Val_Loss improved (7.59643334 --> 6.12938422). Saving model ...


...

Epoch 134/1000 - Train loss: 3.73013325 - Val loss: 3.72983351
INFO: Early stopping counter                     18 of 20                         [Best Val Loss: 3.72765914,                             Val Loss: 3.72983351]


100%|██████████| 8/8 [00:00<00:00,  9.57it/s]


Epoch 135/1000 - Train loss: 3.72845173 - Val loss: 3.73007038
INFO: Early stopping counter                     19 of 20                         [Best Val Loss: 3.72765914,                             Val Loss: 3.73007038]


100%|██████████| 8/8 [00:00<00:00,  9.40it/s]


Epoch 136/1000 - Train loss: 3.75371335 - Val loss: 3.73033398
INFO: Early stopping counter                     20 of 20                         [Best Val Loss: 3.72765914,                             Val Loss: 3.73033398]
INFO: Early stopping DONE!
INFO: Best Val Loss: 3.72765914
Loading best model...


In [10]:
results_uncertainty_aware = indirectUncertaintyAwareHeatmapRegressionPipeline.evaluate(image_paths_test1, landmarks_test1, pixel_spacings_test1)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 150/150 [00:03<00:00, 49.44it/s]


Detection report:
1# Point-to-point error (PE) statistics:
Class               Mean      Median    Std       Min       Max       
----------------------------------------------------------------------
Landmark 0          2.16      1.95      1.46      0.36      13.21     
Landmark 1          2.49      1.86      2.32      0.03      13.28     
Landmark 2          2.81      2.47      1.58      0.20      9.41      
Landmark 3          3.18      2.80      2.03      0.15      10.20     
Landmark 4          4.14      3.78      2.12      0.54      11.39     
Landmark 5          3.98      3.74      2.27      0.24      11.98     
Landmark 6          2.04      1.80      1.13      0.07      4.89      
Landmark 7          1.88      1.72      1.17      0.19      7.28      
Landmark 8          1.75      1.56      1.00      0.12      5.16      
Landmark 9          4.21      3.64      2.74      0.55      19.02     
Landmark 10         3.54      3.35      1.68      0.45      9.58      
Landmark 11       

## Indirect Uncertainty Unaware Heatmap Regression

In [11]:
from landmarker.pipeline import IndirectUncertaintyUnawareHeatmapRegressionPipeline

train_config = {
    "criterion": "mse",
    "optimizer": "adam",
    "lr": lr,
    "batch_size": batch_size,
    "epochs": epochs,
    "lr_scheduler": True,
}

indirectUncertaintyUnawareHeatmapRegressionPipeline = IndirectUncertaintyUnawareHeatmapRegressionPipeline(
    differentiable_decoder="weighted_spatial_mean", model="CoordConvUNet", in_channels_img=1, nb_landmarks=19,
    train_config=train_config, heatmap_decoder=coord_argmax, verbose=True, device=device)

indirectUncertaintyUnawareHeatmapRegressionPipeline.fit(image_paths_train, landmarks_train,
                                        transform=composed_transformd, cache_ds=cache_ds)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 8/8 [00:00<00:00, 15.27it/s]


Epoch 1/1000 - Train loss: 1432.58005371 - Val loss: 349.50663185


100%|██████████| 8/8 [00:00<00:00, 15.27it/s]


Epoch 2/1000 - Train loss: 441.34426168 - Val loss: 328.38031387
INFO: Val_Loss improved (349.50663185 --> 328.38031387). Saving model ...


...

Epoch 257/1000 - Train loss: 28.80437730 - Val loss: 33.67543626
INFO: Early stopping counter                     18 of 20                         [Best Val Loss: 33.44511509,                             Val Loss: 33.67543626]


100%|██████████| 8/8 [00:00<00:00, 15.53it/s]


Epoch 258/1000 - Train loss: 29.39330750 - Val loss: 33.67518425
INFO: Early stopping counter                     19 of 20                         [Best Val Loss: 33.44511509,                             Val Loss: 33.67518425]


100%|██████████| 8/8 [00:00<00:00, 13.12it/s]


Epoch 259/1000 - Train loss: 30.12864227 - Val loss: 33.67101192
INFO: Early stopping counter                     20 of 20                         [Best Val Loss: 33.44511509,                             Val Loss: 33.67101192]
INFO: Early stopping DONE!
INFO: Best Val Loss: 33.44511509
Loading best model...


In [12]:
results_uncertainty_unaware = indirectUncertaintyUnawareHeatmapRegressionPipeline.evaluate(image_paths_test1, landmarks_test1, pixel_spacings_test1)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 150/150 [00:02<00:00, 52.91it/s]

Detection report:
1# Point-to-point error (PE) statistics:
Class               Mean      Median    Std       Min       Max       
----------------------------------------------------------------------
Landmark 0          123.46    122.84    10.00     102.38    146.31    
Landmark 1          112.95    113.62    8.43      82.32     139.87    
Landmark 2          89.53     88.91     7.64      71.85     108.62    
Landmark 3          120.90    119.42    10.27     98.76     144.31    
Landmark 4          56.40     56.65     5.07      35.55     68.80     
Landmark 5          26.53     26.62     3.33      5.92      34.57     
Landmark 6          14.11     14.03     2.70      6.22      20.56     
Landmark 7          14.95     14.94     3.24      2.88      23.62     
Landmark 8          12.69     12.70     2.75      4.58      21.13     
Landmark 9          79.18     79.23     6.88      52.60     98.88     
Landmark 10         39.95     39.92     4.03      18.74     51.52     
Landmark 11       

## Coordinate Regression

In [13]:
from landmarker.pipeline import CoordinateRegressionPipeline
from monai.networks.nets import DenseNet121

train_config = {
    "criterion": "l1",
    "optimizer": "adam",
    "lr": lr,
    "batch_size": batch_size,
    "epochs": epochs,
    "lr_scheduler": True,
}

coordinateRegressionPipeline = CoordinateRegressionPipeline(
    model=DenseNet121(spatial_dims=2, in_channels=1, out_channels=19*2),
    in_channels_img=1, nb_landmarks=19, train_config=train_config, verbose=True, device=device, dim_img=dim_img)

coordinateRegressionPipeline.fit(image_paths_train, landmarks_train,
                                        transform=composed_transformd, cache_ds=cache_ds)

Reading, normalizing and resizing 150 images and landmarks...


100%|██████████| 8/8 [00:00<00:00, 23.18it/s]


Epoch 1/1000 - Train loss: 324.49358622 - Val loss: 323.91516495


100%|██████████| 8/8 [00:00<00:00, 32.02it/s]


Epoch 2/1000 - Train loss: 324.07745972 - Val loss: 322.07958603
INFO: Val_Loss improved (323.91516495 --> 322.07958603). Saving model ...


...

Epoch 170/1000 - Train loss: 16.02510535 - Val loss: 8.84706682
INFO: Early stopping counter                     18 of 20                         [Best Val Loss: 8.65596259,                             Val Loss: 8.84706682]


100%|██████████| 8/8 [00:00<00:00, 32.29it/s]


Epoch 171/1000 - Train loss: 14.85677077 - Val loss: 8.83096230
INFO: Early stopping counter                     19 of 20                         [Best Val Loss: 8.65596259,                             Val Loss: 8.83096230]


100%|██████████| 8/8 [00:00<00:00, 31.56it/s]


Epoch 172/1000 - Train loss: 15.99702911 - Val loss: 8.85800487
INFO: Early stopping counter                     20 of 20                         [Best Val Loss: 8.65596259,                             Val Loss: 8.85800487]
INFO: Early stopping DONE!
INFO: Best Val Loss: 8.65596259
Loading best model...


In [14]:
results_coordinate = coordinateRegressionPipeline.evaluate(image_paths_test1, landmarks_test1, pixel_spacings_test1)

Reading and normalizing 150 images...


100%|██████████| 150/150 [00:17<00:00,  8.60it/s]

Detection report:
1# Point-to-point error (PE) statistics:
Class               Mean      Median    Std       Min       Max       
----------------------------------------------------------------------
Landmark 0          112.22    112.28    4.21      101.73    123.62    
Landmark 1          136.47    136.53    5.06      123.94    150.03    
Landmark 2          144.14    144.21    5.37      130.80    158.59    
Landmark 3          116.74    116.80    4.37      105.85    128.54    
Landmark 4          167.86    167.95    6.24      152.39    184.59    
Landmark 5          187.13    187.23    6.96      169.86    205.77    
Landmark 6          196.39    196.49    7.33      178.20    216.06    
Landmark 7          198.34    198.45    7.40      179.99    218.15    
Landmark 8          198.40    198.51    7.39      180.07    218.19    
Landmark 9          159.48    159.56    5.98      144.60    175.53    
Landmark 10         179.67    179.76    6.69      163.06    197.61    
Landmark 11       